In [ ]:
import tensorflow as tf
import os
import numpy as np
import skimage.io
import sklearn.preprocessing
import sklearn.model_selection
from tqdm import tqdm


def load_data(data_directory):
    directories = []
    labels = []
    for directory in os.listdir(data_directory):
        if os.path.isdir(os.path.join(data_directory, directory)):
            directories.append(os.path.join(data_directory, directory))
            labels.append(directory)

    directories.sort()
    labels.sort()

    #directories = directories[:4]
    #labels = labels[:4]

    encoder = sklearn.preprocessing.LabelEncoder()
    encoder.fit(labels)
    labels = encoder.transform(labels)

    number = min(len(os.listdir(directory)) for directory in directories)

    images = []
    for directory, label in tqdm(list(zip(directories, labels)), ncols=100):
        for filename in os.listdir(directory)[:number]:
            if filename.lower().endswith(".jpg"):
                name = os.path.join(directory, filename)
                temp = skimage.io.imread(name)
                if temp.shape != (256, 256, 3):
                    continue
                images.append((temp, label))

    ret = np.array(images)
    np.random.shuffle(ret)

    images, labels = zip(*ret)
    images, labels = np.array(images), np.array(tf.keras.utils.to_categorical(labels))
    images = images / 255
    return images, labels


def create_model(shape, output):
    model = tf.keras.models.Sequential()
    # model.add(tf.keras.layers.BatchNormalization(input_shape=shape))
    model.add(tf.keras.layers.Conv2D(64, (3, 3), padding="same", strides=(2, 2), activation="relu", input_shape=shape))
    model.add(tf.keras.layers.MaxPooling2D((2, 2)))
    model.add(tf.keras.layers.Conv2D(128, (3, 3), padding="same", strides=(2, 2), activation="relu"))
    model.add(tf.keras.layers.MaxPooling2D((2, 2)))
    model.add(tf.keras.layers.Conv2D(256, (3, 3), padding="same", strides=(2, 2), activation="relu"))
    model.add(tf.keras.layers.MaxPooling2D((2, 2)))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(512))
    model.add(tf.keras.layers.Dropout(0.25))
    model.add(tf.keras.layers.Dense(128))
    model.add(tf.keras.layers.Dropout(0.25))
    model.add(tf.keras.layers.Dense(output, activation="softmax"))
    model.compile(optimizer='adam', loss="categorical_crossentropy", metrics=["accuracy"])
    return model


def main():
    #ROOT_PATH = os.path.dirname(os.path.realpath(__file__))
    #data_directory = os.path.join(ROOT_PATH, "raw", "segmented")
    data_directory = "/content/drive/My Drive/segmented"

    images, labels = load_data(data_directory)
    model = create_model(images[0].shape, labels.shape[1])
    
    trainx, testx, trainy, testy = sklearn.model_selection.train_test_split(images, labels, test_size=0.1, shuffle=True)

    model.fit(x=trainx, y=trainy, batch_size=16, epochs=20, validation_data=(testx, testy))

if __name__ == "__main__":
    np.random.seed(1234)
    main()



100%|████████████████████████████████████████████████████████████| 38/38 [1:05:06<00:00, 102.81s/it]


Epoch 1/20
325/325 [==============================] - 11s 35ms/step - loss: 2.8710 - accuracy: 0.1905 - val_loss: 2.2103 - val_accuracy: 0.3651
Epoch 2/20
325/325 [==============================] - 10s 31ms/step - loss: 1.8074 - accuracy: 0.4485 - val_loss: 1.4999 - val_accuracy: 0.5761
Epoch 3/20
325/325 [==============================] - 11s 33ms/step - loss: 1.2578 - accuracy: 0.6221 - val_loss: 1.3000 - val_accuracy: 0.6107
Epoch 4/20
325/325 [==============================] - 11s 33ms/step - loss: 0.9180 - accuracy: 0.7152 - val_loss: 1.1188 - val_accuracy: 0.6419
Epoch 5/20
325/325 [==============================] - 11s 34ms/step - loss: 0.7176 - accuracy: 0.7716 - val_loss: 1.1739 - val_accuracy: 0.6592
Epoch 6/20
325/325 [==============================] - 11s 34ms/step - loss: 0.5284 - accuracy: 0.8241 - val_loss: 1.1298 - val_accuracy: 0.6817
Epoch 7/20
325/325 [==============================] - ETA: 0s - loss: 0.4445 - accuracy: 0.8568

In [ ]:
import tensorflow as tf
import os
import numpy as np
import skimage.io
import sklearn.preprocessing
import sklearn.model_selection
from tqdm import tqdm
import shutil


def load_data(data_directory):
    directories = []
    labels = []
    for directory in os.listdir(data_directory):
        if os.path.isdir(os.path.join(data_directory, directory)):
            directories.append(os.path.join(data_directory, directory))
            labels.append(directory)

    directories.sort()
    labels.sort()

    #directories = directories[:4]
    #labels = labels[:4]

    number = 3 * min(len(os.listdir(directory)) for directory in directories)

    images = []
    final_labels = []
    for directory, label in tqdm(list(zip(directories, labels)), ncols=100):
        for filename in os.listdir(directory)[:number]:
            if filename.lower().endswith(".jpg"):
                name = os.path.join(directory, filename)
                images.append(name) 
                final_labels.append(label)

    return images, final_labels

def main():
    #ROOT_PATH = os.path.dirname(os.path.realpath(__file__))
    #data_directory = os.path.join(ROOT_PATH, "raw", "segmented")
    data_directory = "/content/drive/My Drive/segmented"

    images, labels = load_data(data_directory)
    
    trainx, testx, trainy, testy = sklearn.model_selection.train_test_split(images, labels, test_size=0.1, shuffle=True)
    print(np.asarray(trainx).shape, np.asarray(testx).shape, np.asarray(trainy).shape, np.asarray(testy).shape)
    exit()

    #shutil.rmtree("/content/drive/My Drive/leafnet_thesis/leaf_train_v3")
    #os.makedirs("/content/drive/My Drive/leafnet_thesis/leaf_train_v3")
    for i, (image, label) in tqdm(enumerate(zip(trainx, trainy)), total=len(trainx)):
        os.makedirs(f"/content/drive/My Drive/leafnet_thesis/leaf_train_v3_v2/{label}", exist_ok=True)
        shutil.copy(image, f"/content/drive/My Drive/leafnet_thesis/leaf_train_v3_v2/{label}/{i}.jpg")

    #shutil.rmtree("/content/drive/My Drive/leafnet_thesis/leaf_valid_v3")
    #os.makedirs("/content/drive/My Drive/leafnet_thesis/leaf_valid_v3")
    for i, (image, label) in tqdm(enumerate(zip(testx, testy)), total=len(testx)):
        os.makedirs(f"/content/drive/My Drive/leafnet_thesis/leaf_valid_v3_v2/{label}", exist_ok=True)
        shutil.copy(image, f"/content/drive/My Drive/leafnet_thesis/leaf_valid_v3_v2/{label}/{i}.jpg")

if __name__ == "__main__":
    np.random.seed(1234)
    main()


  0%|          | 0/12915 [00:00<?, ?it/s]

(12915,) (1436,) (12915,) (1436,)


  1%|          | 112/12915 [01:31<2:41:56,  1.32it/s]

KeyboardInterrupt: ignored

  1%|          | 112/12915 [01:32<2:55:34,  1.22it/s]


In [ ]:
import tensorflow as tf
import os
import numpy as np
import skimage.io
import sklearn.preprocessing
import sklearn.model_selection
from tqdm import tqdm

def preprocess(image):
    return image / 255

def create_model(shape, output):
    model = tf.keras.models.Sequential()
    # model.add(tf.keras.layers.BatchNormalization(input_shape=shape))
    model.add(tf.keras.layers.Conv2D(64, (3, 3), padding="same", strides=(2, 2), activation="relu", input_shape=shape))
    model.add(tf.keras.layers.MaxPooling2D((2, 2)))
    model.add(tf.keras.layers.Conv2D(128, (3, 3), padding="same", strides=(2, 2), activation="relu"))
    model.add(tf.keras.layers.MaxPooling2D((2, 2)))
    model.add(tf.keras.layers.Dropout(0.3))
    model.add(tf.keras.layers.Conv2D(256, (3, 3), padding="same", strides=(2, 2), activation="relu"))
    model.add(tf.keras.layers.MaxPooling2D((2, 2)))
    model.add(tf.keras.layers.Dropout(0.3))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(512))
    model.add(tf.keras.layers.Dropout(0.3))
    model.add(tf.keras.layers.Dense(128))
    model.add(tf.keras.layers.Dropout(0.3))
    model.add(tf.keras.layers.Dense(output, activation="softmax"))
    model.compile(optimizer='sgd', loss="categorical_crossentropy", metrics=["accuracy"])
    return model


def main():
    batch_size = 16
    #ROOT_PATH = os.path.dirname(os.path.realpath(__file__))
    #data_directory = os.path.join(ROOT_PATH, "raw", "segmented")
    #data_directory = "/content/drive/My Drive/segmented"
    model = create_model((256, 256, 3), 33)

    train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=preprocess)
    valid_datagen = tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=preprocess)
    
    train_generator = train_datagen.flow_from_directory(
        directory="/content/drive/My Drive/leafnet_thesis/leaf_train",
        target_size=(256, 256),
        color_mode="rgb",
        batch_size=batch_size,
        class_mode="categorical",
        shuffle=True,
        seed=42
    )

    valid_generator = valid_datagen.flow_from_directory(
        directory="/content/drive/My Drive/leafnet_thesis/leaf_valid",
        target_size=(256, 256),
        color_mode="rgb",
        batch_size=batch_size,
        class_mode="categorical",
        shuffle=True,
        seed=42
    )

    checkpoint = tf.keras.callbacks.ModelCheckpoint("/content/drive/My Drive/leafnet_thesis/models/leafnet.h5", monitor="val_loss", save_best_only=True)
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=3, restore_best_weights=True)
    term_on_nan = tf.keras.callbacks.TerminateOnNaN()

    history = model.fit_generator(
        train_generator,
        steps_per_epoch=train_generator.n // batch_size,
        validation_data=valid_generator,
        validation_steps=valid_generator.n // batch_size,
        epochs=200,
        verbose=1,
        workers=-1,
        callbacks=[checkpoint, early_stopping, term_on_nan]
    )

if __name__ == "__main__":
    np.random.seed(1234)
    main()



In [ ]:
import tensorflow as tf
import os
import numpy as np
import skimage.io
import sklearn.preprocessing
import sklearn.model_selection
from tqdm import tqdm

# def preprocess(image):
#     return image / 255

def create_model(shape, output):
    model = tf.keras.models.Sequential()
    # model.add(tf.keras.layers.BatchNormalization(input_shape=shape))
    model.add(tf.keras.layers.Conv2D(64, (3, 3), padding="same", strides=(2, 2), activation="relu", input_shape=shape))
    model.add(tf.keras.layers.MaxPooling2D((2, 2)))
    model.add(tf.keras.layers.Conv2D(128, (3, 3), padding="same", strides=(2, 2), activation="relu"))
    model.add(tf.keras.layers.MaxPooling2D((2, 2)))
    model.add(tf.keras.layers.Dropout(0.3))
    model.add(tf.keras.layers.Conv2D(256, (3, 3), padding="same", strides=(2, 2), activation="relu"))
    model.add(tf.keras.layers.MaxPooling2D((2, 2)))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(1024))
    model.add(tf.keras.layers.Dropout(0.3))
    model.add(tf.keras.layers.Dense(256))
    model.add(tf.keras.layers.Dropout(0.3))
    model.add(tf.keras.layers.Dense(output, activation="softmax"))
    model.compile(optimizer='sgd', loss="categorical_crossentropy", metrics=["accuracy", tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
    model.summary()
    return model


def main():
    batch_size = 8
    #ROOT_PATH = os.path.dirname(os.path.realpath(__file__))
    #data_directory = os.path.join(ROOT_PATH, "raw", "segmented")
    #data_directory = "/content/drive/My Drive/segmented"
    model = create_model((256, 256, 3), 33)

    train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        rescale=1./255,
        horizontal_flip=True,
        vertical_flip=True,
        rotation_range=45,
        brightness_range=[0.9, 1.1],
        channel_shift_range=20,
        zoom_range=0.15,
        shear_range=0.1,
        fill_mode='nearest',
        #zca_whitening=True
    )
    valid_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
    
    train_generator = train_datagen.flow_from_directory(
        directory="/content/drive/My Drive/leafnet_thesis/leaf_train_v3",
        target_size=(256, 256),
        color_mode="rgb",
        batch_size=batch_size,
        class_mode="categorical",
        shuffle=True,
        seed=42,
        
    )

    valid_generator = valid_datagen.flow_from_directory(
        directory="/content/drive/My Drive/leafnet_thesis/leaf_valid_v3",
        target_size=(256, 256),
        color_mode="rgb",
        batch_size=batch_size,
        class_mode="categorical",
        shuffle=True,
        seed=42
    )

    checkpoint = tf.keras.callbacks.ModelCheckpoint("/content/drive/My Drive/leafnet_thesis/models/leafnet_v3_down_v1.h5", monitor="val_loss", save_best_only=True)
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=3, restore_best_weights=True)
    term_on_nan = tf.keras.callbacks.TerminateOnNaN()

    history = model.fit_generator(
        train_generator,
        steps_per_epoch=train_generator.n // batch_size,
        validation_data=valid_generator,
        validation_steps=valid_generator.n // batch_size,
        epochs=200,
        verbose=1,
        workers=-1,
        callbacks=[checkpoint, early_stopping, term_on_nan]
    )

if __name__ == "__main__":
    np.random.seed(1234)
    main()



Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 128, 128, 64)      1792      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 64, 64, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 128)       73856     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 128)       0         
_________________________________________________________________
dropout (Dropout)            (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 8, 8, 256)         295168    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 4, 4, 256)         0

FileNotFoundError: ignored

In [ ]:
import tensorflow as tf
import os
import numpy as np
import skimage.io
import sklearn.preprocessing
import sklearn.model_selection
from tqdm import tqdm

# def preprocess(image):
#     return image / 255

def create_model(shape, output):
    model = tf.keras.models.Sequential()
    # model.add(tf.keras.layers.BatchNormalization(input_shape=shape))
    model.add(tf.keras.layers.Conv2D(16, (3, 3), padding="same", strides=(2, 2), activation="relu", input_shape=shape))
    model.add(tf.keras.layers.MaxPooling2D((2, 2)))
    model.add(tf.keras.layers.Conv2D(32, (3, 3), padding="same", strides=(2, 2), activation="relu"))
    model.add(tf.keras.layers.MaxPooling2D((2, 2)))
    model.add(tf.keras.layers.Dropout(0.3))
    model.add(tf.keras.layers.Conv2D(64, (3, 3), padding="same", strides=(2, 2), activation="relu"))
    model.add(tf.keras.layers.MaxPooling2D((2, 2)))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(256))
    model.add(tf.keras.layers.Dropout(0.3))
    model.add(tf.keras.layers.Dense(64))
    model.add(tf.keras.layers.Dropout(0.3))
    model.add(tf.keras.layers.Dense(output, activation="softmax"))
    model.compile(optimizer='sgd', loss="categorical_crossentropy", metrics=["accuracy", tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
    model.summary()
    return model


def main():
    batch_size = 8
    #ROOT_PATH = os.path.dirname(os.path.realpath(__file__))
    #data_directory = os.path.join(ROOT_PATH, "raw", "segmented")
    #data_directory = "/content/drive/My Drive/segmented"
    model = create_model((320, 320, 3), 2)

    train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        rescale=1./255,
        horizontal_flip=True,
        vertical_flip=True,
        rotation_range=45,
        brightness_range=[0.9, 1.1],
        channel_shift_range=20,
        zoom_range=0.15,
        shear_range=0.1,
        fill_mode='nearest',
        #zca_whitening=True
    )
    valid_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
    
    train_generator = train_datagen.flow_from_directory(
        directory="/content/drive/My Drive/dataset_irrigation/irrigation_train",
        target_size=(320, 320),
        color_mode="rgb",
        batch_size=batch_size,
        class_mode="categorical",
        shuffle=True,
        seed=42,
        
    )

    valid_generator = valid_datagen.flow_from_directory(
        directory="/content/drive/My Drive/dataset_irrigation/irrigation_valid",
        target_size=(320, 320),
        color_mode="rgb",
        batch_size=batch_size,
        class_mode="categorical",
        shuffle=True,
        seed=42
    )

    checkpoint = tf.keras.callbacks.ModelCheckpoint("/content/drive/My Drive/irrigation.h5", monitor="val_loss", save_best_only=True)
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=3, restore_best_weights=True)
    term_on_nan = tf.keras.callbacks.TerminateOnNaN()

    history = model.fit_generator(
        train_generator,
        steps_per_epoch=train_generator.n // batch_size,
        validation_data=valid_generator,
        validation_steps=valid_generator.n // batch_size,
        epochs=200,
        verbose=1,
        workers=-1,
        callbacks=[checkpoint, early_stopping, term_on_nan]
    )

if __name__ == "__main__":
    np.random.seed(1234)
    main()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 160, 160, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 80, 80, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 40, 40, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 20, 20, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 20, 20, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 10, 10, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0

In [ ]:
import tensorflow as tf
import os
import numpy as np
import skimage.io
import sklearn.preprocessing
import sklearn.model_selection
from tqdm import tqdm

# def preprocess(image):
#     return image / 255

def create_model(shape, output):
    model = tf.keras.models.Sequential()
    # model.add(tf.keras.layers.BatchNormalization(input_shape=shape))
    model.add(tf.keras.layers.Conv2D(32, (3, 3), padding="same", strides=(2, 2), activation="relu", input_shape=shape))
    model.add(tf.keras.layers.MaxPooling2D((2, 2)))
    model.add(tf.keras.layers.Conv2D(64, (3, 3), padding="same", strides=(2, 2), activation="relu"))
    model.add(tf.keras.layers.MaxPooling2D((2, 2)))
    model.add(tf.keras.layers.Dropout(0.3))
    model.add(tf.keras.layers.Conv2D(128, (3, 3), padding="same", strides=(2, 2), activation="relu"))
    model.add(tf.keras.layers.MaxPooling2D((2, 2)))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(512))
    model.add(tf.keras.layers.Dropout(0.3))
    model.add(tf.keras.layers.Dense(128))
    model.add(tf.keras.layers.Dropout(0.3))
    model.add(tf.keras.layers.Dense(output, activation="softmax"))
    model.compile(optimizer='sgd', loss="categorical_crossentropy", metrics=["accuracy", tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
    model.summary()
    return model


def main():
    batch_size = 8
    #ROOT_PATH = os.path.dirname(os.path.realpath(__file__))
    #data_directory = os.path.join(ROOT_PATH, "raw", "segmented")
    #data_directory = "/content/drive/My Drive/segmented"
    model = create_model((128, 128, 3), 33)

    train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        rescale=1./255,
        horizontal_flip=True,
        vertical_flip=True,
        rotation_range=45,
        brightness_range=[0.9, 1.1],
        channel_shift_range=20,
        zoom_range=0.15,
        shear_range=0.1,
        fill_mode='nearest',
        #zca_whitening=True
    )
    valid_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
    
    train_generator = train_datagen.flow_from_directory(
        directory="/content/drive/My Drive/leafnet_thesis/leaf_train_v3",
        target_size=(128, 128),
        color_mode="rgb",
        batch_size=batch_size,
        class_mode="categorical",
        shuffle=True,
        seed=42,
        
    )

    valid_generator = valid_datagen.flow_from_directory(
        directory="/content/drive/My Drive/leafnet_thesis/leaf_valid_v3",
        target_size=(128, 128),
        color_mode="rgb",
        batch_size=batch_size,
        class_mode="categorical",
        shuffle=True,
        seed=42
    )

    checkpoint = tf.keras.callbacks.ModelCheckpoint("/content/drive/My Drive/leafnet_thesis/models/leafnet_v3_down_v2_128.h5", monitor="val_loss", save_best_only=True)
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=3, restore_best_weights=True)
    term_on_nan = tf.keras.callbacks.TerminateOnNaN()

    history = model.fit_generator(
        train_generator,
        steps_per_epoch=train_generator.n // batch_size,
        validation_data=valid_generator,
        validation_steps=valid_generator.n // batch_size,
        epochs=200,
        verbose=1,
        workers=-1,
        callbacks=[checkpoint, early_stopping, term_on_nan]
    )

if __name__ == "__main__":
    np.random.seed(1234)
    main()




Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 64, 64, 32)        896       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 16, 16, 64)        18496     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 8, 8, 64)          0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 8, 8, 64)          0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 4, 4, 128)         73856     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 2, 2, 128)        

In [ ]:
import pandas as pd
s = pd.Series({'c': 1, 'b': 2, 'a':3})
s

In [ ]:
#final_version
import tensorflow as tf
import os
import numpy as np
import skimage.io
import sklearn.preprocessing
import sklearn.model_selection
from tqdm import tqdm

# def preprocess(image):
#     return image / 255

def create_model(shape, output):
    model = tf.keras.models.Sequential()
    # model.add(tf.keras.layers.BatchNormalization(input_shape=shape))
    model.add(tf.keras.layers.Conv2D(64, (3, 3), padding="same", strides=(2, 2), activation="relu", input_shape=shape))
    model.add(tf.keras.layers.MaxPooling2D((2, 2)))
    model.add(tf.keras.layers.Conv2D(128, (3, 3), padding="same", strides=(2, 2), activation="relu"))
    model.add(tf.keras.layers.MaxPooling2D((2, 2)))
    model.add(tf.keras.layers.Dropout(0.3))
    model.add(tf.keras.layers.Conv2D(256, (3, 3), padding="same", strides=(2, 2), activation="relu"))
    model.add(tf.keras.layers.MaxPooling2D((2, 2)))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(1024))
    model.add(tf.keras.layers.Dropout(0.3))
    model.add(tf.keras.layers.Dense(256))
    model.add(tf.keras.layers.Dropout(0.3))
    model.add(tf.keras.layers.Dense(output, activation="softmax"))
    model.compile(optimizer='sgd', loss="categorical_crossentropy", metrics=["accuracy", tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
    model.summary()
    return model


def main():
    batch_size = 16
    #ROOT_PATH = os.path.dirname(os.path.realpath(__file__))
    #data_directory = os.path.join(ROOT_PATH, "raw", "segmented")
    #data_directory = "/content/drive/My Drive/segmented"
    model = create_model((256, 256, 3), 33)

    train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        rescale=1./255,
        horizontal_flip=True,
        vertical_flip=True,
        rotation_range=45,
        brightness_range=[0.9, 1.1],
        channel_shift_range=20,
        zoom_range=0.15,
        shear_range=0.1,
        fill_mode='nearest',
        #zca_whitening=True
    )
    valid_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
    
    train_generator = train_datagen.flow_from_directory(
        directory="/content/drive/My Drive/leafnet_thesis/leaf_train_v3",
        target_size=(256, 256),
        color_mode="rgb",
        batch_size=batch_size,
        class_mode="categorical",
        shuffle=True,
        seed=42,
        
    )

    valid_generator = valid_datagen.flow_from_directory(
        directory="/content/drive/My Drive/leafnet_thesis/leaf_valid_v3",
        target_size=(256, 256),
        color_mode="rgb",
        batch_size=batch_size,
        class_mode="categorical",
        shuffle=True,
        seed=42
    )

    checkpoint = tf.keras.callbacks.ModelCheckpoint("/content/drive/My Drive/leafnet_thesis/models/leafnet_v3_final.h5", monitor="val_loss", save_best_only=True)
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=3, restore_best_weights=True)
    term_on_nan = tf.keras.callbacks.TerminateOnNaN()

    history = model.fit_generator(
        train_generator,
        steps_per_epoch=train_generator.n // batch_size,
        validation_data=valid_generator,
        validation_steps=valid_generator.n // batch_size,
        epochs=200,
        verbose=1,
        workers=-1,
        callbacks=[checkpoint, early_stopping, term_on_nan]
    )

if __name__ == "__main__":
    np.random.seed(1234)
    main()



Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 128, 128, 64)      1792      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 64, 64, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 128)       73856     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 128)       0         
_________________________________________________________________
dropout (Dropout)            (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 8, 8, 256)         295168    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 4, 4, 256)         0